In [1]:
from ctypes import *
import time
import os
import sys
import tempfile 
import re
from socket import socket, gethostbyname, AF_INET, SOCK_DGRAM
import numpy as np

In [3]:
class network_interface(object):
    def __init__(self,partner_ip='<Gatan computer IP>',send_port=2567,receive_port=2568):
        self.partner_ip = partner_ip
        self.send_port = send_port
        self.receive_port = receive_port
        self.here = gethostbyname( '0.0.0.0' ) # for receiving
        self.send_socket = socket( AF_INET, SOCK_DGRAM )
        self.receive_socket = socket( AF_INET, SOCK_DGRAM )
        self.receive_socket.bind( (self.here, self.receive_port) )
        print ("Sending scripts to IP {0}, via port {1}\n".format(partner_ip, self.send_port))

def send_and_wait(script,interface,SIZE=1024):
    interface.send_socket.sendto(script,(interface.partner_ip,interface.send_port))
    print('Script sent.')
    (success,addr) = interface.receive_socket.recvfrom(SIZE)
    if(int(success) == 0):
        print('Script completed successfully.')
        return int(success)
    elif(int(success) == 1):
        print('Script is too long. Reduce size and try again. (See server for printout of max script size.)')
        return int(success)
    else:
        print('Received from Digital Micrograph: {}'.format(str(success)))
        raise Exception('Either script failed or there is a communication error.')  

In [7]:
in1 = network_interface() # connect to Gatan computer

Sending scripts to IP 134.76.90.104, via port 2567



In [8]:
data_folder = '\\\\<Gatan computer local network name>\\<External_Data_Drive>\\<Username>\\' # filepath as seen from external hardware computer; 
data_folder_DM = 'E:\\\\<Username>\\\\' # filepath as seen from Gatan computer
base_name = 'date_experimentname'
sample = 'sample'
option = 'option'
if( not os.path.isdir(data_folder+base_name) ):
    os.mkdir(data_folder+base_name)
if( not os.path.isdir(data_folder+base_name+'\\'+sample) ):
    os.mkdir(data_folder+base_name+'\\'+sample)
if( not os.path.isdir(data_folder+base_name+'\\'+sample+'\\'+option) ):
    os.mkdir(data_folder+base_name+'\\'+sample+'\\'+option)
basedir = data_folder+base_name+'\\'+sample+'\\'+option
basedir_DM = data_folder_DM+base_name+'\\\\'+sample+'\\\\'+option

In [10]:
# requirements for use:
# - Valid survey image and ROI already assigned
# -  SI array size already defined
# -  Signals and other SI settings made
dm_script = '''
// Set Save folder
string saveFolder = %s
string tagPath = "SI:Acquisition:Preferences:Save:Group Save Preferences:Base folder"
GetPersistentTagGroup().TagGroupSetTagAsString(tagPath,saveFolder)

// Start SI
number nSI = %d
for (number i=0; i<nSI; i++ )
{
    SIInvokeButton( "Start/Stop", 1 )
    while(SIIsAcquisitionActive()) yield()
    sleep(0.5)          // Small delay needed to allow DM finish SI acquisition clean-up
}
'''